In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("test_lib_from_postgres_to_clickhouse")
    .config(
        "spark.jars.packages",
        ",".join([
            # clickhouse drivers
            "com.clickhouse:clickhouse-jdbc:0.7.2",
            # postgres drivers
            "org.postgresql:postgresql:42.7.3"
        ])
    )
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.clickhouse#clickhouse-jdbc added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-43a9f4bd-23ab-41be-b826-495dc9080479;1.0
	confs: [default]
	found com.clickhouse#clickhouse-jdbc;0.7.2 in central
	found com.clickhouse#clickhouse-client;0.7.2 in central
	found com.clickhouse#clickhouse-data;0.7.2 in central
	found com.clickhouse#clickhouse-http-client;0.7.2 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.2 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.httpcomponents.client5#httpclient5;5.2.1 in central
	found org.apache.httpcomponents.core5#httpcore5;5.2.1 in central
	found org.apache.commons#commons-compress;1.27.1 in central
	found commons-codec#commons-codec;1.17.1 in central
	found commons-io#commons-io;2.16.1 in central
	found org.apache.commons

In [2]:
from data_transfer_lib import Postgres, ClickHouse, Reader, Writer

# Создание коннектов
postgres_conn = Postgres(
    host="postgres",
    user="postgres_user",
    password="postgres_password",
    database="postgres",
    spark=spark
)

clickhouse_conn = ClickHouse(
    host="clickhouse-server",
    user="default",
    password="1234qwe",
    database="default",
    spark=spark
)

# Чтение из Postgres
reader = Reader(
    connection=postgres_conn,
    db_name="mydb",
    table_name="users"
)
df = reader.start()

# Загрузка в ClickHouse
uploader = Writer(
    connection=clickhouse_conn,
    db_name="analytics",
    table_name="users",
    if_exists=True
)
uploader.start(df=df)

Инициализирован Postgres
Валидация параметров подключения
Инициализирован ClickHouse
Валидация параметров подключения
Инициализирован Reader
Вызван метод Reader._prepare
Получить схему таблицы из PostgreSQL для mydb.users
Вызван метод SchemaValidator.validate_source_to_spark
Вызван метод Reader.start для mydb.users
Параметры: {}
Инициализирован Writer
Вызван метод Writer._prepare
Получить схему таблицы из ClickHouse для analytics.users
Получена схема target таблицы analytics.users
Вызван метод Writer.start для analytics.users
Параметры: {}
Вызван метод SchemaValidator.validate_spark_to_target
Данные успешно загружены (заглушка)
